In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('handlandmarks.csv')

In [3]:
x = df.drop('label', axis=1).values
y = df['label'].values

In [4]:
#split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
#reshape the data for the CNN model
X_train_cnn = X_train.reshape(-1, 42, 1)
X_test_cnn = X_test.reshape(-1, 42, 1)


In [ ]:
#define the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(42, 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(26, activation='softmax'))

In [ ]:
#compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train the CNN model
history = cnn_model.fit(X_train_cnn, tf.keras.utils.to_categorical(y_train), epochs=25, batch_size=32, validation_data=(X_test_cnn, tf.keras.utils.to_categorical(y_test)))

In [ ]:
y_pred_cnn = cnn_model.predict(X_test_cnn)
y_pred_class_cnn = np.argmax(y_pred_cnn, axis=1)
accuracy_cnn = accuracy_score(y_test, y_pred_class_cnn)
print(f'Test accuracy (CNN): {accuracy_cnn:.3f}')

In [ ]:
# cnn_model.save('cnn_model.h5')

In [13]:
#define the SVM model
svm_model = SVC(kernel='rbf', C=1, gamma=0.1)
# svm_model = SVC(kernel='rbf', C=10, gamma=0.01)
# svm_model = SVC(kernel='poly', degree=2, C=1, gamma=0.1)
# svm_model = SVC(kernel='rbf', C=100, gamma=0.1)
svm_model.fit(X_train, y_train)


SVC(C=1, gamma=0.1)

In [ ]:
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f'Test accuracy (SVM): {accuracy_svm:.3f}')

In [ ]:
#define the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


In [ ]:
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Test accuracy (Random Forest): {accuracy_rf:.3f}')


In [ ]:
#create a confusion matrix for each model
conf_mat_cnn = confusion_matrix(y_test, y_pred_class_cnn)
conf_mat_svm = confusion_matrix(y_test, y_pred_svm)
conf_mat_rf = confusion_matrix(y_test, y_pred_rf)



In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_cnn, annot=True, cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix (CNN)')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_svm, annot=True, cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix (SVM)')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat_rf, annot=True, cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix (Random Forest)')
plt.show()